In [1]:
import sys
sys.path.append("../model")
from SklearnModels import Regressor

import datetime
import numpy as np
import pandas as pd
import math
import glob
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [3]:
import torch
from LSTMModel import Net

#### Load data

In [4]:
# read in training samples
file = '../../data/2006NDVI_LCin_all.csv'
mydata = pd.read_csv(file, index_col=0)

# replace NA with 0
cmydata = mydata.fillna(0.0)

# drop rows where uvi == 0
mmidat = cmydata[cmydata.uvi != 0.0].reset_index(drop=True)


# split into train & test sets
test_ratio = 0.2
train, test = train_test_split(mmidat, test_size=test_ratio, random_state=12)
# split dependent(Y) and indenpendent(X) variable
d_var = 'uvi'
ind_vars = ['upre', 'upvi', 'u2pvi', 'u3pvi', 'u4pvi', 'u5pvi']
xTrain, yTrain = train[ind_vars], train[d_var] 
xTest, yTest = test[ind_vars],test[d_var]

# generate subset
train_subset = train.sample(frac=0.1)
train_subset.shape
xTrain_sub, yTrain_sub = train_subset[ind_vars], train_subset[d_var]

/att/gpfsfs/home/appmgr/app/jupyterhub/ilab/ilab-kernel/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


### Train CNN

In [5]:
# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
x, y = xTrain_sub.to_numpy(), yTrain_sub.to_numpy()

x = x.reshape(x.shape[0], 1, x.shape[1])
x.shape, y.shape

((618515, 1, 6), (618515,))

In [7]:
# numpy array to pytorch tensor
x_train_ts = torch.from_numpy(x).type(torch.float32)
y_train_ts = torch.from_numpy(y).type(torch.float32)

In [8]:
net = Net()
net = net.to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
loss_func = torch.nn.MSELoss()

In [9]:
## Start training
import torch.utils.data as Data
from torch.autograd import Variable

In [10]:
##
BATCH_SIZE = 128
EPOCH = 10

torch_dataset = Data.TensorDataset(x_train_ts, y_train_ts)

loader = Data.DataLoader(
    dataset=torch_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, num_workers=2,)

In [11]:
for epoch in range(EPOCH):
    epoch_loss = 0
    net = net.train()
    for step, (batch_x, batch_y) in enumerate(loader):
        b_x = Variable(batch_x).to(device)
        b_y = Variable(batch_y).to(device)
        
        prediction = net(b_x)
        loss = loss_func(prediction, b_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    print(f'Epoch {epoch+0:03}: | Train Loss: {epoch_loss/len(loader):.5f}') 

Epoch 000: | Train Loss: 1087639.04398
Epoch 001: | Train Loss: 930052.67642
Epoch 002: | Train Loss: 926459.74283
Epoch 003: | Train Loss: 923750.97275
Epoch 004: | Train Loss: 918919.41614
Epoch 005: | Train Loss: 919072.29718
Epoch 006: | Train Loss: 917081.01055
Epoch 007: | Train Loss: 915537.57687
Epoch 008: | Train Loss: 911080.80283
Epoch 009: | Train Loss: 908713.50164


In [ ]:
torch.save(net, 'cnn_checkpoint.chk')

In [12]:
pred = net(x_train_ts.to(device))
pred

tensor([[2347.5918],
        [2321.6440],
        [2266.5288],
        ...,
        [2239.6323],
        [2434.3872],
        [2258.9814]], device='cuda:0', grad_fn=<AddmmBackward>)

In [13]:
y_train_ts

tensor([1877., 2224., 1942.,  ...,  124., 3459., 1761.])